# Рекомендательные системы на основе SVD алгоритма

**Collaborative Filtering** - подход, основанный на отношениях между пользователями и товаром, при этом нет никакой информации о пользователях или товарах. 

* Пример явного взаимодействия: рейтинг между пользователям и товарами.
* Неявного: клик, просмотр или покупка не очевидны с точки зрения рейтинга.

**User-item matrix** (матрица пользователь-товар) - это таблица, в которой каждая строка представляет собой пользователя, каждый столбец - товар, а каждая ячейка содержит информацию о взаимодействии между пользователем и товаром. Например, в ячейке может быть указано количество раз, которое пользователь купил товар, или оценка, которую пользователь поставил товару. User-item matrix используется в рекомендательных системах для определения предпочтений пользователей и предложения им наиболее подходящих товаров.

Далее, $R$ - user-item matrix.

**Singular Value Decomposition (SVD)** 

Можно применить [матричную факторизацию](https://translated.turbopages.org/proxy_u/en-ru.ru.0832cecd-64b5506e-d08cf623-74722d776562/https/en.wikipedia.org/wiki/Matrix_factorization_(recommender_systems)). Часто, матричная факторизация применяется в области уменьшения размерности, где мы пытаемся уменьшить количество элементов, сохраняя при этом соответсующую информацию. Так дело обстоит с [основным компонентным анализом (PCA)](https://ru.wikipedia.org/wiki/Метод_главных_компонент) и очень похожим [разложение по сингулярному значению (SVD)](https://ru.wikipedia.org/wiki/Сингулярное_разложение).

$$R_{m \times n} = U_{m \times m} \Sigma_{m \times n} V^{T}_{n \times n} $$

где $\Sigma$  — диагональная матрица с неотрицательными сингулярными числами, $U$ и $V$ - ортогональные матрицы и $U^T U = I$.

При этом, матрица $R$ - очень большая и разряженная. 

**Метод главных компонент ([principal component analysis (PSA)](https://ru.wikipedia.org/wiki/Метод_главных_компонент))** — один из основных способов уменьшить размерность данных, потеряв наименьшее количество информации. 
Вычисление главных компонент может быть сведено к вычислению SVD разложения матрицы данных или к вычислению собственных векторов и собственных значений ковариационной матрицы исходных данных. 

Математическое содержание метода главных компонент — это SVD разложение ковариационной матрицы $R$, то есть представление пространства данных в виде суммы взаимно ортогональных собственных подпространств $R$, а самой матрицы $R$ — в виде линейной комбинации ортогональных проекторов на эти подпространства с коэффициентами $\lambda_i$. 

Оставив $d$-первых (главных) компонент ($\lambda_i$) в SVD разложении, получим наиболее точное приближение к $R$ по норме Фробениуса.  
 
$ R_{m \times n} = U_{m \times n} V_{n \times n} $

$ U = U_{m \times m} \Sigma_{m \times d} $ 

$ V = V_{d \times n}$

$U$ и $V$ - взаимно ортогональные собственные подпространства $R$.

Тогда $i$-aя строка $U$ - это представление $i$-го пльзователя, а $j$-ый столбец $V$ - представленте $j$-го товара. Их отношение (рейтинг) выражается с помощью скалярного произведения:

$$R[i][j] = \langle U[i, :], V[:, j] \rangle $$

Матрицы $ U $ и $ Q $ найдем с помощью градиентного спуска. 

В качестве $ Loss $ возьмем $ RMSE $:

$$ 
    RMSE = \frac{1}{\lvert D \rvert} \sum_{(i,j) \in D} (\hat{R}_{i, j} - R_{i, j})^2 = 
    \frac{1}{\lvert D \rvert} \sum_{(i, j) \in D} (U_{i} \cdot V_{j} - R_{i, j})^2
$$

Посчитав градиенты по $ U $ и по $ V $, получим формулы обновления весов:

<!-- $\nabla RMSE_p = \frac{d}{dp}RMSE = \frac{1}{\lvert D \rvert} \cdot 2 \cdot \sum_{(u,m) \in D} (\hat{r}_{u,m} - r_{u,m}) \cdot q $

$ \nabla RMSE_q = \frac{d}{dq}RMSE = \frac{1}{\lvert D \rvert} \cdot 2 \cdot \sum_{(u,m) \in D} (\hat{r}_{u,m} - r_{u,m}) \cdot p $ -->

$\begin{cases}
    U[i][k] = U[i][k] - \frac{2}{\lvert D \rvert} \cdot \sum_{(i,j) \in D} (\hat{R}_{i,j} - R_{i,j}) \cdot V[k][i]
    \\
    V[k][i] = V[k][i] - \frac{2}{\lvert D \rvert} \cdot \sum_{(i,j) \in D} (\hat{R}_{i,j} - R_{i,j}) \cdot U[i][k]
\end{cases}
\text{   ,   } k \in {1, d} $

In [1]:
import numpy as np
import pandas as pd
%matplotlib inline

In [2]:
k = 10 # максимальная оценка

movies = ['Фантазия', 'ВАЛЛ-И', 'Пиноккио', 'Бемби' , 'Шрэк', 'Дамбо', 'Спасатели', 'Геркулес', 'Кунг-фу Панда']
m_movies = len(movies)

users = ['Андрей', 'Аня', 'Алиса', 'Ваня', 'Леша', 'Оксана', 'Саша', 'Паша', 'Сеня', 'Гриша']        
n_users = len(users)

RANDOMSEED = 42
np.random.seed(42)

N = np.random.randint(20, 60)
ind_movies = [np.random.randint(0, m_movies) for _ in range(N)]
ind_users = [np.random.randint(0, n_users) for _ in range(N)]
rating = [np.random.randint(1, k) for _ in range(N)]

In [3]:
print(ind_movies)

[3, 7, 4, 6, 2, 6, 7, 4, 3, 7, 7, 2, 5, 4, 1, 7, 5, 1, 4, 0, 5, 8, 0, 2, 6, 3, 8, 2, 4, 2, 6, 4, 8, 6, 1, 3, 8, 1, 8, 4, 1, 3, 6, 7, 2, 0, 3, 1, 7, 3, 1, 5, 5, 3, 5, 1, 1, 3]


In [4]:
print(ind_users)

[7, 6, 8, 7, 4, 1, 4, 7, 9, 8, 8, 0, 8, 6, 8, 7, 0, 7, 7, 2, 0, 7, 2, 2, 0, 4, 9, 6, 9, 8, 6, 8, 7, 1, 0, 6, 6, 7, 4, 2, 7, 5, 2, 0, 2, 4, 2, 0, 4, 9, 6, 6, 8, 9, 9, 2, 6, 0]


In [5]:
print(rating)

[4, 4, 5, 7, 7, 4, 7, 3, 6, 2, 9, 5, 6, 4, 7, 9, 7, 1, 1, 9, 9, 4, 9, 3, 7, 6, 8, 9, 5, 1, 3, 8, 6, 8, 9, 4, 1, 1, 4, 7, 2, 3, 1, 5, 1, 8, 1, 1, 2, 2, 6, 7, 5, 1, 1, 3, 2, 5]


In [6]:
def get_user_item_matrix(n_users, m_movies, ind_users, ind_movies, rating):
    R = [[0] * m_movies for _ in range(n_users)]
    N = len(ind_users)
    for i in range(N):
        R[ind_users[i]][ind_movies[i]] = rating[i]
    R = np.array(R)
    return R

R = get_user_item_matrix(n_users, m_movies, ind_users, ind_movies, rating)

pd.DataFrame(R, users, movies)

,Фантазия,ВАЛЛ-И,Пиноккио,Бемби,Шрэк,Дамбо,Спасатели,Геркулес,Кунг-фу Панда
Андрей,0,1,5,5,0,9,7,5,0
Аня,0,0,0,0,0,0,8,0,0
Алиса,9,3,1,1,7,0,1,0,0
Ваня,0,0,0,0,0,0,0,0,0
Леша,8,0,7,6,0,0,0,2,4
Оксана,0,0,0,3,0,0,0,0,0
Саша,0,2,9,4,4,7,3,4,1
Паша,0,2,0,4,1,0,7,9,6
Сеня,0,7,1,0,8,5,0,9,0
Гриша,0,0,0,1,5,1,0,0,8


In [7]:
def MSE(R, U, V):
    mse = 0
    for ind in range(N):
        i = ind_users[ind]
        j = ind_movies[ind]
        mse += ( R[i][j] - np.dot(U[i,:], V[:,j]) )**2 / N
    return mse

In [8]:
def SVD(ind_users, ind_movies, R, d, step, n_iters):
    
    # инициализация матриц разложения
    U = np.random.rand(R.shape[0], d)
    V = np.random.rand(d, R.shape[1])

    start_mse = MSE(R, U, V)
    
    for n in range(n_iters):
        ind = np.random.randint(0, N)
        i = ind_users[ind]
        j = ind_movies[ind]
        
        for k in range(0, d):
            U[i, k] = U[i, k] + step * (R[i][j] - np.dot(U[i, :], V[:, j])) * V[k, j] 
            V[k, j] = V[k, j] + step * (R[i][j] - np.dot(U[i, :], V[:, j])) * U[i, k] 

        mse = MSE(R, U, V)
    return U, V, start_mse, mse

In [9]:
U, V, start_mse, mse = SVD(ind_users, ind_movies, R, 3, 0.1, 3000) 

In [10]:
U

array([[ 1.59507097e+00,  1.72612654e+00,  1.52339541e+00],
       [ 1.37564070e+00,  2.37155072e+00,  5.65473065e-01],
       [ 2.64594856e+00,  3.13163441e-01,  9.83491686e-03],
       [ 6.33529711e-01,  5.35774684e-01,  9.02897701e-02],
       [ 2.12139021e+00, -4.78727555e-04,  2.69803740e+00],
       [ 9.63724489e-02,  8.21234247e-01,  9.69383422e-01],
       [ 1.37629513e+00,  9.17312819e-01,  1.09177106e+00],
       [ 8.77410525e-01,  1.97726763e+00,  1.23516733e+00],
       [ 6.82853447e-01,  2.92020164e+00, -2.30772170e-01],
       [ 1.01539658e+00, -9.17608418e-01,  7.63367674e-01]])

In [11]:
V

array([[ 3.16227766,  0.69283784,  3.16227766,  0.22109154,  3.16227766,
         3.16227766,  0.33523404, -0.06493694,  3.16227766],
       [ 1.57063954,  2.01477781,  2.50489965,  0.9593538 ,  2.10017969,
         2.72906909,  3.15316639,  3.1622776 ,  1.33026677],
       [ 0.4943566 , -1.91429227,  2.15246234,  2.3155225 , -0.21616623,
         0.38415414,  0.66757026,  0.82395472,  0.48178968]])

In [12]:
R[1][2]

0

In [13]:
np.dot(U[1,:], V[:,2])

11.507813915439094

In [14]:
start_mse, mse

(23.33405679536876, 8.911327611150677)

In [15]:
cap_R = np.zeros((R.shape[0], R.shape[1]))

for i in range(n_users):
    for j in range(m_movies):
        cap_R[i][j] = round(np.dot(U[i,:], V[:,j]), 1)
cap_R

array([[ 8.5,  1.7, 12.6,  5.5,  8.3, 10.3,  7. ,  6.6,  8.1],
       [ 8.4,  4.6, 11.5,  3.9,  9.2, 11. ,  8.3,  7.9,  7.8],
       [ 8.9,  2.4,  9.2,  0.9,  9. ,  9.2,  1.9,  0.8,  8.8],
       [ 2.9,  1.3,  3.5,  0.9,  3.1,  3.5,  2. ,  1.7,  2.8],
       [ 8. , -3.7, 12.5,  6.7,  6.1,  7.7,  2.5,  2.1,  8. ],
       [ 2.1, -0.1,  4.4,  3.1,  1.8,  2.9,  3.3,  3.4,  1.9],
       [ 6.3,  0.7,  9. ,  3.7,  6. ,  7.3,  4.1,  3.7,  6.1],
       [ 6.5,  2.2, 10.4,  5. ,  6.7,  8.6,  7.4,  7.2,  6. ],
       [ 6.6,  6.8,  9. ,  2.4,  8.3, 10. ,  9.3,  9. ,  5.9],
       [ 2.1, -2.6,  2.6,  1.1,  1.1,  1. , -2. , -2.3,  2.4]])

In [16]:
pd.DataFrame(cap_R, users, movies)

,Фантазия,ВАЛЛ-И,Пиноккио,Бемби,Шрэк,Дамбо,Спасатели,Геркулес,Кунг-фу Панда
Андрей,8.5,1.7,12.6,5.5,8.3,10.3,7.0,6.6,8.1
Аня,8.4,4.6,11.5,3.9,9.2,11.0,8.3,7.9,7.8
Алиса,8.9,2.4,9.2,0.9,9.0,9.2,1.9,0.8,8.8
Ваня,2.9,1.3,3.5,0.9,3.1,3.5,2.0,1.7,2.8
Леша,8.0,-3.7,12.5,6.7,6.1,7.7,2.5,2.1,8.0
Оксана,2.1,-0.1,4.4,3.1,1.8,2.9,3.3,3.4,1.9
Саша,6.3,0.7,9.0,3.7,6.0,7.3,4.1,3.7,6.1
Паша,6.5,2.2,10.4,5.0,6.7,8.6,7.4,7.2,6.0
Сеня,6.6,6.8,9.0,2.4,8.3,10.0,9.3,9.0,5.9
Гриша,2.1,-2.6,2.6,1.1,1.1,1.0,-2.0,-2.3,2.4
